In [8]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Pitching Airfoil 

In [9]:
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from time import time

# Jax 64-bit
from jax.config import config
config.update("jax_enable_x64", True)

C:\Users\tryfo\AppData\Local\Temp\ipykernel_11840\2405514880.py:8: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config


# Dynamical Systems

In [10]:
from ICARUS.Systems.first_order_system import NonLinearSystem
from ICARUS.Systems.second_order_system import SecondOrderSystem
from ICARUS.Systems.integrate import BackwardEulerIntegrator, ForwardEulerIntegrator, RK4Integrator, RK45Integrator, CrankNicolsonIntegrator, GaussLegendreIntegrator, NewmarkIntegrator

In [11]:
def plot_results(x_data, t_data):
    # Plot the results
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    ax.set_title("Simulation Results")
    for key in list(x_data.keys()):
        # Compute cap so that we plot 1000 points at most
        cap = max(1, int(len(t_data[key]) / 1000))
        ax.plot(t_data[key][::cap], x_data[key][:,0][::cap] * 180/np.pi, label=key)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Angle (deg)")
    ax.legend()


def test_all_integrators(system,x0, t0, t_end, dt0, compare_with_scipy = False):
    if isinstance(system, SecondOrderSystem):
        system2 = system.convert_to_first_order()
        newmark = NewmarkIntegrator(dt0, system, gamma=0.5, beta=0.25)
    else:
        system2 = system
    
    integrator_rk4 = RK4Integrator(dt0, system2)
    integrator_feuler = ForwardEulerIntegrator(dt0, system2)
    integrator_beuler = BackwardEulerIntegrator(dt0, system2)
    integrator_rk45 = RK45Integrator(dt0, system2)
    integrator_gauss_legendre = GaussLegendreIntegrator(dt0, system2, n = 4)
    integrator_crank_nicolson = CrankNicolsonIntegrator(dt0, system2)

    integrators = [
        integrator_feuler,
        integrator_rk4,
        integrator_rk45,
        integrator_beuler,
        integrator_gauss_legendre,
        # integrator_crank_nicolson
     ]
    
    if isinstance(system, SecondOrderSystem):
        integrators.append(newmark)

    # Simulate the system using all the integrators
    x_data = {}
    t_data = {}


    for integrator in integrators:
        print(f"Simulating using {integrator.name} integrator")
        time_s = time()
        t_data[integrator.name], x_data[integrator.name] = integrator.simulate(x0, t0, t_end)
        time_e = time()
        print(f"\tSimulated using {integrator.name} integrator")
        print(f"\tTime taken: {time_e - time_s} seconds")

    if compare_with_scipy:
        print("Simulating using scipy RK45")
        time_s = time()
        sol = solve_ivp(system, [t0, t_end], x0, method='RK45', t_eval=np.linspace(0, t_end, 1000),  rtol=1e-6, atol=1e-6)
        t_data["scipy_rk45"] = sol.t
        x_data["scipy_rk45"] = sol.y.T
        time_e = time()
        print(f"\tSimulated using scipy RK45")
        print(f"\tTime taken: {time_e - time_s} seconds")

    plot_results(x_data, t_data)
    return x_data, t_data

In [12]:
# Define a simple m-c-k system
L = 8
I = 1
R = 2.0
K = 10
d= .5
rho = 1.225
U = 0.5
c = 2


theta_0 = 1 * np.pi / 180
crank_y = R * np.cos(theta_0)
crank_x = R * np.sin(theta_0)
y_equilibrium = np.sqrt(L**2 - R**2 * np.cos(theta_0)**2) + R*np.sin(theta_0)

DL = np.sqrt(
    (
        y_equilibrium - R*np.sin(theta_0)
     )**2 + R**2*np.cos(theta_0)**2
    ) - L
print(DL)

0.0


In [13]:
y_equilibrium

7.78095014878384

In [15]:
from ICARUS.Database import DB
DB.load_data()
airf = DB.get_airfoil("NACA0024")
polar = DB.foils_db.get_polars("NACA0024", "Xfoil")
cl, cd, cm = polar.get_aero_coefficients(polar.reynolds_nums[0], 0.)
a_0 = polar.get_reynolds_zero_lift_angle(polar.reynolds_nums[0])
print(a_0)
a_0 = 0

ERROR:root:Airfoil 0024 not found in DB2D or EXTERNAL DB


Airfoil 0024 not found in DB2D or EXTERNAL DB
Error in interpolating the airfoil data
Lower xs:
[0.000000e+00 6.870000e-05 7.181000e-04 1.878700e-03 3.843900e-03
 6.322100e-03 8.814300e-03 1.181410e-02 1.782690e-02 2.154460e-02
 2.756840e-02 3.359530e-02 3.962520e-02 4.565700e-02 5.169020e-02
 5.973630e-02 6.979530e-02 7.985710e-02 8.991900e-02 9.998280e-02
 1.100465e-01 1.251442e-01 1.402415e-01 1.553399e-01 1.704383e-01
 1.855357e-01 2.006342e-01 2.157315e-01 2.308300e-01 2.459284e-01
 2.610257e-01 2.761242e-01 2.912235e-01 3.113556e-01 3.314900e-01
 3.516258e-01 3.717628e-01 3.919010e-01 4.120401e-01 4.321793e-01
 4.523206e-01 4.724628e-01 4.926062e-01 5.127506e-01 5.328960e-01
 5.530424e-01 5.731888e-01 5.933372e-01 6.134868e-01 6.336375e-01
 6.537892e-01 6.739418e-01 6.940945e-01 7.142490e-01 7.344044e-01
 7.545605e-01 7.747174e-01 7.948748e-01 8.150318e-01 8.351902e-01
 8.553490e-01 8.755080e-01 8.956673e-01 9.158267e-01 9.259060e-01
 9.359855e-01 9.460661e-01 9.561460e-01 9.6622

In [16]:
xs = {}
ts = {}

In [17]:
def y_input(t:float) -> jnp.ndarray:
    return y_equilibrium + 0.2 * jnp.sin(2 * jnp.pi / 10 *t)


from jax.debug import print as jprint
def f(t:float, x: jnp.ndarray) -> jnp.ndarray:
    y = y_input(t)
    th = x[0]
    v = x[1]                            # x' = v

    sin = jnp.sin(th)
    cos = jnp.cos(th)
        
    a = -(
        L * R * K * y * cos / jnp.sqrt(R**2 - y * (2 * R * sin - y)) - R * K * y * cos
    )

    # # Theodorsen method for the lift coefficient
    l_ec = 1/2
    ksi = (l_ec -3/4)*c

    th_th = th
    v_th =  v
    a_th =  a

    W_eff2 = (
        (-U * jnp.cos(0) + jnp.sin(th_th) * ksi * v_th)**2 + (U * jnp.sin(0) + jnp.cos(-th_th) * ksi * v_th)**2
    )       
    W_eff = jnp.sqrt(W_eff2) 
    a_eff = -jnp.arctan(
        (
            U * jnp.sin(0) + jnp.cos(th_th)* ksi * v_th
        )/(
            -U * jnp.cos(0) + jnp.sin(th_th)* ksi * v_th
        )
    ) - th_th

    A1 = 0.165
    A2 = 0.335
    b1 = 0.0455
    b2 = 0.3
    
    y1 = x[2]                           
    y2 = x[3]

    C_k_epi_a = a_eff * ( 1 - A1 - A2) + y1 + y2 
    cl = 2* jnp.pi * (C_k_epi_a - a_0) + jnp.pi*c / (2*W_eff2) * (-W_eff * v_th - (1/2 - l_ec)*a_th *c)
    cm = (4*l_ec -1)*jnp.pi/2 * C_k_epi_a - jnp.pi/ (2*W_eff2) * (-(3/4 - l_ec)*W_eff * v_th * c - 1/4 * (9/8 + 4*l_ec**2 -4*l_ec)*a_th *c**2)
    
    a_y1 = - (
        b1 * 2 * W_eff /c * y1 - b1*A1 *2*W_eff/c * a_eff
    )
    a_y2 = - (
        b2 * 2 * W_eff /c * y2 - b2*A2 *2*W_eff/c * a_eff
    )
    cl, cd, cm = polar.get_aero_coefficients(polar.reynolds_nums[0], a_eff * 180 / jnp.pi)

    # PRINT
    # jprint("t= {time}", time = t)
    # jprint("W_eff = {x}", x=W_eff)
    # jprint('Th = {x}', x=th*180/jnp.pi)
    # jprint('A_eff = {x}', x=a_eff*180/jnp.pi)
    # jprint('a = {x}', x=a)
    # jprint('v = {x}', x=v)
    # jprint("CL = {x}",x=cl)
    # jprint('cm = {x}', x=cm)
    # jprint('')

    # # Q is the aerodynamic moment
    # Q = 0.5 * rho * W_eff**2 * c * (cm)
    Q = 0.5 * rho * W_eff**2 * c * (cl *d + cm)
    return jnp.array([
        v,              # x' = v
        a +0* Q/I,        # v' = a = f(t, x)/ m
        a_y1,           # y1
        a_y2            # y2
    ])

# Create the system
system = NonLinearSystem(f)

# Test the integrators
dt = 1e-6*5
x0 = jnp.array([theta_0, 0.0])
x0 = jnp.array([theta_0, 0.0,0.,0.])
x_data, t_data = test_all_integrators(system, x0, 0.0, 40, dt, compare_with_scipy = True)

Simulating using Forward Euler integrator
	Simulated using Forward Euler integrator
	Time taken: 6.83450722694397 seconds
Simulating using RK4 integrator
	Simulated using RK4 integrator
	Time taken: 21.13376235961914 seconds
Simulating using RK45 integrator
	Simulated using RK45 integrator
	Time taken: 10.084479331970215 seconds
Simulating using Backward Euler integrator
	Simulated using Backward Euler integrator
	Time taken: 44.038068532943726 seconds
Simulating using Gauss-Legendre integrator
	Simulated using Gauss-Legendre integrator
	Time taken: 67.22828817367554 seconds
Simulating using scipy RK45
	Simulated using scipy RK45
	Time taken: 1.3761770725250244 seconds


In [18]:
xs['pol_cl_cm'] = x_data['Gauss-Legendre']
ts['pol_cl_cm'] = t_data['Gauss-Legendre']

In [19]:
xs3 = {k:v for k,v in xs.items() if k in ['theo_cl_cm', 'pol_cl_cm',]}
ts3 = {k:v for k,v in ts.items() if k in ['theo_cl_cm', 'pol_cl_cm',]}

plot_results(xs3, ts3)

In [20]:
xs.keys()

dict_keys(['pol_cl_cm'])

In [21]:
x = x_data[list(x_data.keys())[0]][:,0].tolist()
t = t_data[list(x_data.keys())[0]][:].tolist()

from scipy.fft import fft, fftfreq

# Perform FFT
fft_result = fft(x)
freqs = fftfreq(len(x), t[1] - t[0])

# Find dominating frequencies
n = len(x)
mask = freqs > 0
fft_result_magnitude = 2.0/n * np.abs(fft_result)
dominant_freqs = freqs[mask][np.argsort(fft_result_magnitude[mask])][::-1]

# Plot original time series data
# plt.figure(figsize=(10, 5))
# plt.subplot(1, 2, 1)
# plt.plot(t, x)
# plt.title("Original Time Series Data")
# plt.xlabel("Time")
# plt.ylabel("Amplitude")

# Plot FFT result
plt.subplot(1, 2, 2)
plt.plot(freqs[mask], fft_result_magnitude[mask])
plt.title("FFT Analysis")
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.xlim(0, max(dominant_freqs)*2)  # Adjust x-axis limit for better visualization

# Mark dominating frequencies
# for freq in dominant_freqs:
#     plt.axvline(x=freq, color='r', linestyle='--')

plt.show()

# Print dominating frequencies
# print("Dominant Frequencies:")
# for freq in dominant_freqs:
#     print("{:.2f} Hz".format(freq))

# Simple Mass-Damper System

In [ ]:
# Define a simple m-c-k system
m = 1.0
c = 0.1
k = 1.0


def f(t:float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array([
        x[1],                            # x' = v
        -c /m * x[1] - k/m * x[0]        # v' = a = -c/m * v - k/m * x
    ])

# Create the system
system = NonLinearSystem(f)

# Test the integrators
test_all_integrators(system, jnp.array([1.0, 0.0]), 0.0, 100.0, 0.0001, compare_with_scipy = True)

# Higher Order

In [ ]:
x = jnp.array([0.1, 0.0])
y = y_input(0)

th = x[0]
v = x[1]                            # x' = v

a = jnp.sin(th)* R * K * y * (
    1 - L / jnp.sqrt(
        R**2 - 2*R*y*jnp.cos(th) + y**2
        )
) / I 

print(jnp.sin(th)* R * K * y)

In [ ]:
jnp.sin(th)

# Second Order Systems

In [ ]:
# Define a 2nd order system 
m1 = 1.0
c1 = 0.1
k1 = 1.0

m2 = 1.0
c2 = 0.1
k2 = 1.0

def M(t,x):
    return jnp.array(
    [
        [m1, 0],
        [0, m2]
    ])
# M = jnp.array([m])
def C(t,x):
    return jnp.array(
    [
        [0.023, 1.024], #[c1, 0],
        [-0.364, 3.31]  #[0, c2]
    ])
# C = jnp.array([c])

def f_int(t,x):
    return jnp.array(
    [
        [1.97, 0.034],  #[k1, -k1],
        [0.034, 3.95]   #[-k1, k1 + k2]
    ])
# f_int = jnp.array([k])

def f_ext(t: float, x: jnp.ndarray) -> jnp.ndarray:
    return jnp.array([
        0.078,# 0.0,
        10*0.466*jnp.sin(t)
    ])
# f_ext = lambda t, x: jnp.array([0.0])

system = SecondOrderSystem(M, C, f_int, f_ext)

In [ ]:
# Test the integrators
test_all_integrators(system, jnp.array([0.0, 0.0, 0.0, 0.0]), 0.0, 100.0, 1e-4, compare_with_scipy = True)

# Another Example

In [ ]:
# Create two finite elements
E = 1#235e9
A = 1#np.pi * (0.3**2) / 4
L = 1.0
I = 1#30e-2
rho = 1#5490


In [ ]:
num_elems  = 6

l = L / num_elems
Cm = rho*A
Ck = E*I
# element mass and stiffness matrices
m = np.array([
    # [140,       0,               0,             70,         0,              0           ], # x1
    [           156,             22*l,                    54,             -13*l       ], # y1
    [           22*l,            4*l**2,                  13*l,           -3*l**2     ], # Theta1
    # [70,         0,              0,             140,        0,              0           ], # x2
    [           54,              13*l,                    156,            -22*l       ], # y2
    [           -13*l,          -3*l**2,                  -22*l,          4*l**2      ]  # Theta2
]) * rho * A * l / 420


k = np.array([
    # [E*A/l,     0,               0,             -E*A/l,     0,              0           ], # x1
    [            12*E*I/L**3,     6*E*I/L**2,                -12*E*I/L**3,   6*E*I/L**2  ],  # y1
    [            6*E*I/L**2,      4*E*I/L,                   -6*E*I/L**2,    2*E*I/L     ],  # Theta1
    # [-E*A/l,    0,               0,             E*A/l,      0,              0           ], # x2
    [           -12*E*I/L**3,    -6*E*I/L**2,                 12*E*I/L**3,    -6*E*I/L**2 ], # y2
    [           6*E*I/L**2,      2*E*I/L,                     -6*E*I/L**2,    4*E*I/L     ]  # Theta2
])

p = 1 / num_elems / l
q = 0.0
f_load = np.array(
       [
            # 0*l/2,      # x1
            p*l/2,      # y1
            p*l**2/12,  # Theta1
            # 0*l/2,      # x2
            p*l/2,      # y2
            -p*l**2/12  # Theta2
       ]
)
# construct global mass and stiffness matrices
dof_per_elem = 2
M = np.zeros((dof_per_elem*(num_elems+1), dof_per_elem*(num_elems+1)))
K = np.zeros((dof_per_elem*(num_elems+1), dof_per_elem*(num_elems+1)))
C = np.zeros((dof_per_elem*(num_elems+1), dof_per_elem*(num_elems+1)))
f_ext = np.zeros((dof_per_elem*(num_elems+1)))

# for each element, change to global coordinates
for i in range(num_elems):
    M_temp = np.zeros((dof_per_elem*(num_elems+1),dof_per_elem*(num_elems+1)))
    K_temp = np.zeros((dof_per_elem*(num_elems+1),dof_per_elem*(num_elems+1)))
    F_temp = np.zeros((dof_per_elem*(num_elems+1)))

    # start = i:dof_per_elem
    # end = (i+2)*dof_per_elem
    M_temp[i*dof_per_elem  :  (i+2)*dof_per_elem, i*dof_per_elem  :  (i+2)*dof_per_elem] = m
    K_temp[i*dof_per_elem  :  (i+2)*dof_per_elem, i*dof_per_elem  :  (i+2)*dof_per_elem] = k
    F_temp[i*dof_per_elem  :  (i+2)*dof_per_elem] = f_load
    M += M_temp
    K += K_temp
    f_ext += F_temp

restrained_dofs = [1, 0, ]
# remove the fixed degrees of freedom
for dof in restrained_dofs:
    for i in [0,1]:
        M = np.delete(M, dof, axis=i)
        K = np.delete(K, dof, axis=i)
        C = np.delete(C, dof, axis=i)
    f_ext = np.delete(f_ext, dof)          

system = SecondOrderSystem(M, C, K, f_ext)
# test_all_integrators(system, jnp.zeros((2*M.shape[0])), 0.0, 100.0, 1e-4, compare_with_scipy = False)

In [ ]:
from scipy.linalg import eigh

evals, evecs = eigh(K,M)
freq = np.sqrt(evals)
print(evals)
print(freq[0])

In [ ]:
# # Newmark integrator
dt0 = 0.0001
newmark = NewmarkIntegrator(dt0, system, gamma=0.5, beta=0.25)
x0 = jnp.zeros((2*M.shape[0]))
t0 = 0.0
t_end = 30.
t_data, x_data = newmark.simulate(x0, t0, t_end)

In [ ]:
import matplotlib.pyplot as plt

clip = 1000

fig, ax = plt.subplots(2, 1, figsize=(10, 10))

# Clip the data to only sample 1000 points
clip = jnp.maximum(1, int(len(t_data) / 1000))

ax[0].set_title("Displacement of node 1")
ax[0].plot(t_data[::clip], x_data[:,0][::clip], label="x1")
ax[0].plot(t_data[::clip], x_data[:,1][::clip], label="y1")
ax[0].plot(t_data[::clip], x_data[:,2][::clip], label="theta1")
ax[0].legend()

ax[1].set_title("Displacement of node 2")
ax[1].plot(t_data[::clip], x_data[:,3][::clip], label="x2")
ax[1].plot(t_data[::clip], x_data[:,4][::clip], label="y2")
ax[1].plot(t_data[::clip], x_data[:,5][::clip], label="theta2")
ax[1].legend()

In [ ]:
# FFt of the displacement
from scipy.fft import fft, fftfreq
x = x_data[:,1].tolist()
t = t_data.tolist()
T = t[1] - t[0]

N = len(x)
yf = fft(x)
xf = fftfreq(N, T)[:N//2]
# Remove the DC component
yf[0] = 0

# Find the peak 
idx = np.argmax(np.abs(yf[0:N//2]))
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
# Add a vertical line at the peak
ax.axvline(xf[idx], color='r', linestyle='--')
ax.set_title("FFT of the displacement")
ax.set_xlabel("Frequency (Hz)")
# Add text to the plot
ax.text(xf[idx], 2.0/N * np.abs(yf[idx]), f"Peak at {xf[idx]*(2*np.pi)} Hz", verticalalignment='bottom')